In [5]:
from ipywidgets import interact, interactive, fixed, interact_manual, IntSlider
import ipywidgets as widgets

In [3]:
def f(x):
    return x

In [4]:

interact(f, x=10);

interactive(children=(IntSlider(value=10, description='x', max=30, min=-10), Output()), _dom_classes=('widget-…

In [8]:
i = IntSlider(min=-10, max=30, step=1, value=10)
i

IntSlider(value=10, max=30, min=-10)

In [9]:
i.value

3

In [10]:
i.value = 9